# TP 2

# Imports

## imports base

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import matplotlib
import plotly as py
import plotly.express as px 
import matplotlib.ticker as ticker
plt.style.use('default')
import plotly.offline as opy
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
from datetime import datetime
from datetime import timedelta

import math

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import log_loss

In [3]:
!pip3 install category_encoders

import category_encoders as ce

/bin/sh: 1: pip3: not found


In [4]:
!pip3 install bayesian-optimization

from bayes_opt import BayesianOptimization

/bin/sh: 1: pip3: not found


### Catboost

In [5]:
#Ejecutar si no funciona el import
!pip3 install catboost

from catboost import CatBoostClassifier, Pool, cv

/bin/sh: 1: pip3: not found


### Xgboost

In [6]:
!pip3 install xgboost

import xgboost as xgb

/bin/sh: 1: pip3: not found


### Lightgbm

In [7]:
!pip3 install lightgbm

import lightgbm as lgb
#from lightgbm import LGBMClassifier

/bin/sh: 1: pip3: not found


### Ramdom forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

### Perceptron

In [9]:
from sklearn.linear_model import Perceptron

from sklearn.neural_network import MLPRegressor

# Lectura de datos

In [10]:
entrenamiento = pd.read_csv('Training.csv')
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
ID                                    16947 non-null int64
Region                                16947 non-null object
Territory                             16947 non-null object
Pricing, Delivery_Terms_Quote_Appr    16947 non-null int64
Pricing, Delivery_Terms_Approved      16947 non-null int64
Bureaucratic_Code_0_Approval          16947 non-null int64
Bureaucratic_Code_0_Approved          16947 non-null int64
Submitted_for_Approval                16947 non-null int64
Bureaucratic_Code                     16947 non-null object
Account_Created_Date                  16947 non-null object
Source                                16947 non-null object
Billing_Country                       16947 non-null object
Account_Name                          16947 non-null object
Opportunity_Name                      16947 non-null object
Opportunity_ID                        16947 non-null int6

In [11]:
testeo = pd.read_csv('Test.csv')
testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 51 columns):
ID                                    2551 non-null int64
Region                                2551 non-null object
Territory                             2551 non-null object
Pricing, Delivery_Terms_Quote_Appr    2551 non-null int64
Pricing, Delivery_Terms_Approved      2551 non-null int64
Bureaucratic_Code_0_Approval          2551 non-null int64
Bureaucratic_Code_0_Approved          2551 non-null int64
Submitted_for_Approval                2551 non-null int64
Bureaucratic_Code                     2551 non-null object
Account_Created_Date                  2551 non-null object
Source                                2551 non-null object
Billing_Country                       2551 non-null object
Account_Name                          2551 non-null object
Opportunity_Name                      2551 non-null object
Opportunity_ID                        2551 non-null int64
Sales_Contract_

In [12]:
#hemisferios = pd.read_csv('hemisferios.csv')
#hemisferios = convertirATipo(hemisferios,['Billing_Country','hemisferio'],'category')
#hemisferios.info()

In [13]:
#Prueba de supuestos
supuestos = pd.read_csv('EvaluacionTemporal.csv', index_col='Opportunity_ID')
supuestos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 1 columns):
Target    1567 non-null int64
dtypes: int64(1)
memory usage: 24.5 KB


# Preprocesamiento de ambos DF

## Modificaciones de oportunidad

In [14]:
def convertirATiempo(df,lista):
    for columna in lista:
        df[columna] = pd.to_datetime(df[columna])
    return df

def convertirATipo(df,lista,tipo):
    for columna in lista:
        df[columna] = df[columna].astype(tipo)
    return df

def convertirAFlotante(df,lista):
    for columna in lista:
        df[columna] = df[columna].apply(lambda x: None if (x == "None" or x == "Other") else float(x))
    return df

def establecerTipos(df):
    """cambia los tipos de las columnas"""
    
    #fechas
    listaDeColumnasFechas = ['Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Last_Modified_Date','Delivery_Year','Planned_Delivery_End_Date','Planned_Delivery_Start_Date']
    df = convertirATiempo(df,listaDeColumnasFechas)
    #enteros
    listaDeColumnasEnteras = ['ID','Opportunity_ID']
    convertirATipo(df,listaDeColumnasEnteras,'int')
    #decimales
    listaDeColumnasEnteras = ['Price','ASP','ASP_(converted)','Total_Taxable_Amount','Total_Amount']
    convertirAFlotante(df,listaDeColumnasEnteras)
    #booleanos
    listaDeColumnasBooleanas = ['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved','Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved']
    convertirATipo(df,listaDeColumnasBooleanas,'bool')
    #categoricas
    listaDeColumnasCategoricas = ['Product_Category_B','Product_Family','Region','Territory','Bureaucratic_Code','Source ','Billing_Country','Account_Name','Opportunity_Name']
    convertirATipo(df,listaDeColumnasCategoricas,'category')
    return df

In [15]:
def agregarHemisferios(df):
    #df.reset_index()
    df = df.merge(hemisferios,how='left',on='Billing_Country')
    df['hemisferio'] = df['hemisferio'].apply(lambda x: 'Desconocido' if (x != 'Norte' and x != 'Sur') else x)
    convertirATipo(df,['hemisferio','Billing_Country'],'category')
    #df.set_index('Opportunity_ID')
    return df

In [16]:
def eliminarColumnasSinImportancia(df):
    #"Last_Activity" y "Actual_Delivery_Date" son puros nulos
    #"Submitted_for_Approval" puros valores nulos
    #"Prod_Category_A" tiene todos sus valores iguales
    df = df.drop(['Submitted_for_Approval','Last_Activity','Actual_Delivery_Date','Prod_Category_A'], axis=1)
    return df

def agregarColumnas(df):
    df['Closed'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': True, 'Negotiation':False,'Proposal':False,'Qualification':False})#REVISAR
    df['Won'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': False, 'Negotiation':False,'Proposal':False,'Qualification':False})
    convertirATipo(df,['Stage'],'category')
    return df

def correccionDeTerritorios(df):
    #se refiere a diferentes partes de usa
    df = df.replace({'Territory':{'Central US':'USA','NE America':'USA', 'NW America':'USA','SW America':'USA','SE America':'USA'}})
    #
    df = df.replace({'Territory':{'KSA':'Saudi Arabia'}})
    #No se ha encontrado la dorma de los siguientes datos 'Kosovo','South America', 'South East Asia', 'UAE (Dubai)', 'UAE (Others)' (en el caso de kosovo no lo grafica)
    return df

def preparacionDelDataFrame(df,entrenamiento=True):
    df = eliminarColumnasSinImportancia(df)
    df = establecerTipos(df)
    df = correccionDeTerritorios(df)
    df.rename(columns = {'Source ':'Source'}, inplace = True)
    #df = agregarHemisferios(df)#desactivado desde el readcsv
    if (entrenamiento):
        df = agregarColumnas(df)
    return df

## Por oportunidad

In [17]:
def desacerseDeNulosDateTime(df,columna):
    df[columna] = df[columna].apply(lambda x: datetime.max if (x is None or x is np.NaN or x is pd.NaT) else x)
    return df

def descomposicionDeFechas(df,lista):
    """se nesesitan enteros no fechas"""
    for columna in lista:
        anio = columna + '_anio'
        df[anio] = df[columna].apply(lambda x: x if x is None else x.year)
        mes = columna + '_mes'
        df[mes] = df[columna].apply(lambda x: x if x is None else x.month)
        dia = columna + '_dia'
        df[dia] = df[columna].apply(lambda x: x if (x is None) else x.day)
    df = df.drop(lista, axis=1)
    return df

def establecerEstacion(df,columnaFecha):
    estacion = columnaFecha + '_Estacion'
    df[estacion] = 'Desconocido'
    df['temporalEstaciones'] = df[columnaFecha].apply(lambda x: 'Desconocido' if x is None else estaciones(x,True))
    df.loc[df['hemisferio'] == 'Norte' , estacion] = df['temporalEstaciones']
    df['temporalEstaciones'] = df[columnaFecha].apply(lambda x: 'Desconocido' if x is None else estaciones(x,False))
    df.loc[df['hemisferio'] == 'Sur' , estacion] = df['temporalEstaciones']
    df = df.drop(['temporalEstaciones'], axis=1)
    convertirATipo(df,[estacion],'category')
    return df

def estaciones(fecha,norte):
    if (norte):
        estaciones = ['primavera','verano','otoño','invierno']
    else:
        estaciones = ['otoño','invierno','primavera','verano']
    if (fecha.month == 12 and fecha.day >= 21 or fecha.month <= 3 and fecha.day < 21):
        return estaciones[3]
    elif (fecha.month == 9 and fecha.day >= 21 or fecha.month > 9):
        return estaciones[2]
    elif (fecha.month == 6 and fecha.day >= 21 or fecha.month > 6):
        return estaciones[1]
    elif (fecha.month == 3 and fecha.day >= 21 or fecha.month > 3):
        return estaciones[0]
    else:
        return 'Desconocido'

def diferenciaEntreFechas(df,colFechaInicial,colFechaFinal):
    df['TiempoDeDiferencia'] = (df[colFechaFinal] - df[colFechaInicial])
    df['TiempoDeDiferencia'] = pd.to_numeric(df['TiempoDeDiferencia'].dt.days, downcast='integer')#revisar si se conviete totalmente a dias
    df = df.drop([colFechaInicial,colFechaFinal], axis=1)
    return df

In [18]:
def dataEntrenamiento(df):
  data = df.groupby('Opportunity_ID').agg({'Won':'sum','Closed':'sum',
                                           'modificaciones':'sum',
                                           #'Won':'sum','modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           #'Price':'mean',#Demaciados nulos
                                           #'Quote_Expiry_Date':'max',
                                           #'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           #'Pricing, Delivery_Terms_Approved':'sum',
                                           #'Bureaucratic_Code_0_Approval':'sum',
                                           #'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           #'Total_Taxable_Amount':'sum',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Account_Type':'first',
                                           'Delivery_Quarter':'first',
                                           'Product_Type':'first',
                                           #'Product_Category_B':'first',
                                           'Billing_Country':'first',
                                           'Planned_Delivery_End_Date':'max',
                                           'Planned_Delivery_Start_Date':'min',
                                           'Delivery_Terms':'first',
                                           'Product_Family':'first',
                                           #'ASP_(converted)_Currency':'first',
                                           'TRF':'mean'
                                           #'hemisferio':'first'
                                           })#hay que seguir agregando cosas que son importantes
  return data

def dataTesteo(df):
  data = df.groupby('Opportunity_ID').agg({'modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           #'Price':'mean',#Demaciados nulos
                                           #'Quote_Expiry_Date':'max',
                                           #'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           #'Pricing, Delivery_Terms_Approved':'sum',
                                           #'Bureaucratic_Code_0_Approval':'sum',
                                           #'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           #'Total_Taxable_Amount':'sum',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Account_Type':'first',
                                           'Delivery_Quarter':'first',
                                           'Product_Type':'first',
                                           #'Product_Category_B':'first',
                                           'Billing_Country':'first',
                                           'Planned_Delivery_End_Date':'max',
                                           'Planned_Delivery_Start_Date':'min',
                                           'Delivery_Terms':'first',
                                           'Product_Family':'first',
                                           #'ASP_(converted)_Currency':'first',
                                           'TRF':'mean'
                                           #'hemisferio':'first'
                                           })#hay que seguir agregando cosas que son importantes
  return data  

def agrupacionDeOportunidades(df,entrenamiento=True):
  df['modificaciones'] = 1
  if (entrenamiento):
    data = dataEntrenamiento(df)
    #Solo me interesa saber si se cerro con exito
    #data['Closed'] = data['Closed'].agg(lambda x: 1 if x > 0 else 0)
    data['Won'] = data['Won'].agg(lambda x: 1 if x > 0 else 0)
    #data['Lost'] = data['Closed'] - data['Won']#si es won es closed pero si es closed y no es won es perdido
    #data['Lost'] = data['Lost'].agg(lambda x: 1 if x > 0 else 0)

    #Los casos NO cerrados del set de entrenamiento se desconocen si tendran exito o no
    #por lo que englobarlos como triunfos o fracasos es un error deben ser retirados del set
    data = data[data['Closed'] > 0]
    data = data.drop(['Closed'], axis=1)
  else:
    data = dataTesteo(df)

  #data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: None if x == 0 else x)
  #data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: None if x == 0 else x)
  #data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: None if x == 0 else x)
  #data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: None if x == 0 else x)
  """
  data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: 1 if x > 0 else 0)
  data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: 1 if x > 0 else 0)
  #data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: 1 if x > 0 else 0)
  #data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: 1 if x > 0 else 0)

  #Condensando columnas
  data['Pricing, Delivery_Terms_Condensado'] = data['Pricing, Delivery_Terms_Approved'] - data['Pricing, Delivery_Terms_Quote_Appr']
  data['Pricing, Delivery_Terms_Condensado'] = data['Pricing, Delivery_Terms_Condensado'].agg(lambda x: 1 if x == 0 else 0)
  data = data.drop(['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved'], axis=1)

  data['Bureaucratic_Code_0_Condensado'] = data['Bureaucratic_Code_0_Approved'] - data['Bureaucratic_Code_0_Approval']
  data['Bureaucratic_Code_0_Condensado'] = data['Bureaucratic_Code_0_Condensado'].agg(lambda x: 1 if x == 0 else 0)
  data = data.drop(['Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved'], axis=1)
  #data = data.replace({'Won':})
  data['aprobacion'] = data['Pricing, Delivery_Terms_Condensado'] + data['Bureaucratic_Code_0_Condensado']
  data['aprobacion'] = data['aprobacion'].agg(lambda x: 1 if x > 1 else 0)
  data = data.drop(['Pricing, Delivery_Terms_Condensado','Bureaucratic_Code_0_Condensado'], axis=1)

  """

  #categoricas
  #listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Billing_Country',
  #                              'Delivery_Quarter','Product_Type','Product_Category_B',
  #                              'Delivery_Terms','Product_Family','ASP_(converted)_Currency']
  listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Billing_Country',
                                'Delivery_Quarter','Product_Type',
                                'Delivery_Terms','Product_Family']
  convertirATipo(data,listaDeColumnasCategoricas,'category')

  #data = establecerEstacion(data,'Opportunity_Created_Date')
  #data = establecerEstacion(data,'Quote_Expiry_Date')
  #data = desacerseDeNulosDateTime(data,'Quote_Expiry_Date')#si no hay fecha de expiracion se pre supone que es infinita

  listaDeValoresDeFechas = ['Last_Modified_Date','Opportunity_Created_Date',
                            'Account_Created_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
  data = descomposicionDeFechas(data,listaDeValoresDeFechas)

  #data['Diferencia_Amount'] = data['Total_Amount'] - data['Total_Amount'].mean()
  #data['Diferencia_Amount'] = data['Diferencia_Amount'].agg(lambda x: -x if x<0 else x)

  #data['Total_Amount'] = data['Total_Amount'].agg(lambda x: math.log(x,10))
  #enteros
  #listaDeColumnasEnteras = ['Quote_Expiry_Date_anio','Quote_Expiry_Date_mes','Quote_Expiry_Date_dia']
  #convertirATipo(data,listaDeColumnasEnteras,'int')
  #data = agregarHemisferios(data)

  #data = data.drop(['hemisferio'], axis=1)

  #data = diferenciaEntreFechas(data,'Planned_Delivery_Start_Date','Planned_Delivery_End_Date')

  return data

#Uso las mismas categoricas en todos lados
listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Billing_Country',
                              'Delivery_Quarter','Product_Type',
                              'Delivery_Terms','Product_Family']


## Categoricas a enteros (caso de que el sistema predictor no reconozca category)

In [19]:
def convertirCategoryEnEnteros(df1,df2,columna):
    encoder = ce.BinaryEncoder(cols=[columna])
    encoder.fit(df1)
    #encoder.fit(df2)
    df1 = encoder.transform(df1)
    df2 = encoder.transform(df2)
    return df1,df2

# Ejecución de preparación

## Ejecución por modificaciones de oportunidad

In [20]:
entrenamientoProcesado = preparacionDelDataFrame(entrenamiento)
entrenamientoProcesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 50 columns):
ID                                    16947 non-null int64
Region                                16947 non-null category
Territory                             16947 non-null object
Pricing, Delivery_Terms_Quote_Appr    16947 non-null bool
Pricing, Delivery_Terms_Approved      16947 non-null bool
Bureaucratic_Code_0_Approval          16947 non-null bool
Bureaucratic_Code_0_Approved          16947 non-null bool
Bureaucratic_Code                     16947 non-null category
Account_Created_Date                  16947 non-null datetime64[ns]
Source                                16947 non-null category
Billing_Country                       16947 non-null category
Account_Name                          16947 non-null category
Opportunity_Name                      16947 non-null category
Opportunity_ID                        16947 non-null int64
Sales_Contract_No                     169

In [21]:
testeoProcesado = preparacionDelDataFrame(testeo,False)
testeoProcesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 47 columns):
ID                                    2551 non-null int64
Region                                2551 non-null category
Territory                             2551 non-null object
Pricing, Delivery_Terms_Quote_Appr    2551 non-null bool
Pricing, Delivery_Terms_Approved      2551 non-null bool
Bureaucratic_Code_0_Approval          2551 non-null bool
Bureaucratic_Code_0_Approved          2551 non-null bool
Bureaucratic_Code                     2551 non-null category
Account_Created_Date                  2551 non-null datetime64[ns]
Source                                2551 non-null category
Billing_Country                       2551 non-null category
Account_Name                          2551 non-null category
Opportunity_Name                      2551 non-null category
Opportunity_ID                        2551 non-null int64
Sales_Contract_No                     2551 non-null objec

## Ejecución por oportunidad

In [22]:
agrupacionEntrenamiento = agrupacionDeOportunidades(entrenamientoProcesado)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 28 columns):
Won                                 9791 non-null int64
modificaciones                      9791 non-null int64
Total_Amount                        9791 non-null float64
Region                              9791 non-null category
Territory                           9791 non-null category
ASP_(converted)                     9791 non-null float64
Account_Type                        9791 non-null category
Delivery_Quarter                    9791 non-null category
Product_Type                        9791 non-null category
Billing_Country                     9791 non-null category
Delivery_Terms                      9791 non-null category
Product_Family                      9791 non-null category
TRF                                 9791 non-null float64
Last_Modified_Date_anio             9791 non-null int64
Last_Modified_Date_mes              9791 non-null int64
Last_Modified_Date_dia

In [23]:
agrupacionTesteo = agrupacionDeOportunidades(testeoProcesado,False)
agrupacionTesteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 27 columns):
modificaciones                      1567 non-null int64
Total_Amount                        1567 non-null float64
Region                              1567 non-null category
Territory                           1567 non-null category
ASP_(converted)                     1567 non-null float64
Account_Type                        1567 non-null category
Delivery_Quarter                    1567 non-null category
Product_Type                        1567 non-null category
Billing_Country                     1567 non-null category
Delivery_Terms                      1567 non-null category
Product_Family                      1567 non-null category
TRF                                 1567 non-null float64
Last_Modified_Date_anio             1567 non-null int64
Last_Modified_Date_mes              1567 non-null int64
Last_Modified_Date_dia              1567 non-null int64
Opportunity_Create

## Separación de resultados de entrenamiento

In [24]:
entrenamientoResultado = agrupacionEntrenamiento[['Won']]
entrenamientoResultado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 1 columns):
Won    9791 non-null int64
dtypes: int64(1)
memory usage: 153.0 KB


In [25]:
agrupacionEntrenamiento = agrupacionEntrenamiento.drop(['Won'], axis=1)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 27 columns):
modificaciones                      9791 non-null int64
Total_Amount                        9791 non-null float64
Region                              9791 non-null category
Territory                           9791 non-null category
ASP_(converted)                     9791 non-null float64
Account_Type                        9791 non-null category
Delivery_Quarter                    9791 non-null category
Product_Type                        9791 non-null category
Billing_Country                     9791 non-null category
Delivery_Terms                      9791 non-null category
Product_Family                      9791 non-null category
TRF                                 9791 non-null float64
Last_Modified_Date_anio             9791 non-null int64
Last_Modified_Date_mes              9791 non-null int64
Last_Modified_Date_dia              9791 non-null int64
Opportunity_Created_Da

# Pruebas

## Prueba supuestamete bien (arbitraria con un error aprox de +0.04 en logloss)

In [26]:
def pruebaArbitraria():
    serieDiferenciasDeSupuestos = dataResultante['Target'] - supuestos['Target']
    serieDiferenciasDeSupuestos = serieDiferenciasDeSupuestos.agg(lambda x: abs(x))
  #valores a vencer
  #0.29328282239325076 0.066288348418729 0.25746523730152193
  #0.31706383931591736 0.05312227841294053 0.23048270740543753 ramdom forest
  #0.289665541459807 0.07066249882454302 0.26582418780942985 catboost
  #0.29650158086984807 0.06760957650563981 0.2600184157048108 //0.42
    print(serieDiferenciasDeSupuestos.mean(),serieDiferenciasDeSupuestos.var(),serieDiferenciasDeSupuestos.std())
    return log_loss(supuestos['Target'],dataResultante['Target'])#predijo 0.4774430257846605 mientras el resultado dio 0.43117
#pruebaArbitraria()
#0.45236516666571225

In [27]:
#dataResultante['Target'].agg(lambda x: 1 if x < 0.55 and x > 0.45 else 0).value_counts()

In [28]:
#dataResultante['Target'].agg(lambda x: 1 if x > 0.95 or x < 0.05 else 0).value_counts()

# Modelos

## Funciones de resultados

In [29]:
def generarResultado(df,prediccion):
    resultado = df.copy(True)
    resultado['Target'] = prediccion
    resultado = resultado[['Target']]
    resultado.to_csv('resultado.csv')
    return resultado


def auxiliarUnificarValor(arreglo):
    return arreglo[:,0]

## Catboost

### Grid search

In [30]:
def GridSearchCatboost(train_data_o,train_labels,cat_features):
  """clf = CatBoostClassifier()
  params = {'iterations': [500],
            'depth': [4, 5, 6],
            #'loss_function': ['Logloss', 'CrossEntropy'],
            'l2_leaf_reg': np.logspace(-10, 1, 50),
            'leaf_estimation_iterations': [10],
            'learning_rate': [0.5,0.25,0.05],
            'rsm' : [0.7,0.5]
            #'eval_metric': ['Accuracy'],
            #'use_best_model': ['True'],
            #'logging_level':['Silent'],
            #'random_seed': [42]
          }
  scorer = make_scorer(accuracy_score)
  clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)
  clf_grid.fit(X_train_enc, y_train,cat_features)
  best_param = clf_grid.best_params_
  return best_param"""
  model = CatBoostClassifier(loss_function='Logloss')
  #model = CatBoostClassifier(loss_function='CrossEntropy')

  train_data = train_data_o
  # convert categorical columns to integers
  for header in cat_features:
    train_data[header] = train_data[header].astype('category').cat.codes

  grid = {'learning_rate': list(np.arange(0.05,0.51,0.05)),
          'depth': list(range(6,13,2)),
          'l2_leaf_reg': list(np.arange(2,5.1,0.5)),
          'iterations': list(range(10,101,15)),
          'rsm':list(np.arange(0.1,1.1,0.1))
          }

  grid_search_result = model.grid_search(grid, 
                                        X=train_data, 
                                        y=train_labels,
                                        cv=10,
                                        plot=True)
  
  return grid_search_result

In [31]:
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
#                      'Account_Type','Delivery_Quarter','Billing_Country']
#comentado para evitar ejecucion erronea
#valores = GridSearchCatboost(agrupacionEntrenamiento,entrenamientoResultado,listaDeColumnasCategoricas)
#valores

### Bayesian

In [32]:
def catboost_classifier(learning_rate, depth, l2_leaf_reg,iterations,rsm):
    params = {
        "loss_function": "Logloss",
        #"eval_metric" : "AUC", 
        "depth" : int(depth),
        "l2_leaf_reg" : l2_leaf_reg,
        "learning_rate" : learning_rate,
        #"random_state" : 42,
        "rsm" : rsm,
        "iterations": int(iterations)
        #"logging_level" : "Silent",
        #"thread_count": 24,
        #"num_boost_round": int(num_boost_round)
    }
    X_train = agrupacionEntrenamiento
    y_train = entrenamientoResultado
    categorical_features = listaDeColumnasCategoricas
    #['Region','Territory','Product_Type','Product_Category_B',
    #                        'Account_Type','Delivery_Quarter','Billing_Country']

    train_data = Pool(data=X_train, label=y_train, 
                               cat_features=categorical_features)
    
    cv_result = cv(train_data,
                       params,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['train-Logloss-mean'].iloc[-1]

def bayesianoCatboost():
  grid = {'learning_rate': (0.01,0.10),
          'depth': (8,14),
          'l2_leaf_reg': (0.1,3.0),
          'iterations': (10,100),
          'rsm':(0.01,1.0)
          }
  catboostBO = BayesianOptimization(catboost_classifier, grid)

  return catboostBO.maximize(n_iter=15, init_points=2)

In [33]:
#bayesianoCatboost()
#catboost_classifier(0.05,10,3.5,20,0.5)

### Predicción

In [34]:
def catboost(train_data,train_labels,cat_features,eval_data):
  # Inicializador de CatBoostClassifier
  model = CatBoostClassifier(iterations=100,
                             learning_rate=0.4,#0.5
                             depth=7,#8
                             l2_leaf_reg=4,#3
                             rsm=0.7,#0.5
                             #random_strength=0.75,
                             grow_policy='SymmetricTree'#'SymmetricTree', 'Depthwise' y 'Lossguide'
                             #random_seed=2
                             #use_best_model=True,#Requiere data de evalucion
                             #eval_set=supuestos
                             )
  #parametros importantes por agregar: l2_leaf_reg, rsm
  #para tipos de category: cat_features, one_hot_max_size

  # Entrenamiento del modelo
  #model.fit(train_data, train_labels, cat_features)

  eval_dataset = Pool(data=eval_data,
                      label=supuestos[['Target']],
                      cat_features=cat_features)
  model.fit(train_data, train_labels, cat_features, use_best_model=True, eval_set=eval_dataset)

  # obtener prediccion
  #preds_class = model.predict(eval_data)

  # obtener probabilidad de prediccion
  preds_proba = model.predict_proba(eval_data)

  # obtener RawFormulaVal de la prediccion
  #preds_raw = model.predict(eval_data, prediction_type='RawFormulaVal')

  #return resultado(eval_data,preds_proba)
  importancia = model.get_feature_importance(type= "FeatureImportance",prettified=True)
  return preds_proba,importancia

In [35]:
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
#                      'Account_Type','Delivery_Quarter','Billing_Country',
#                      'Delivery_Terms','Product_Family']
res,importancia = catboost(agrupacionEntrenamiento.copy(deep=True),entrenamientoResultado.copy(deep=True),listaDeColumnasCategoricas,agrupacionTesteo.copy(deep=True))
res

0:	learn: 0.5478415	test: 0.5552760	best: 0.5552760 (0)	total: 126ms	remaining: 12.5s
1:	learn: 0.4906782	test: 0.5177977	best: 0.5177977 (1)	total: 175ms	remaining: 8.55s
2:	learn: 0.4630350	test: 0.4867127	best: 0.4867127 (2)	total: 215ms	remaining: 6.96s
3:	learn: 0.4414192	test: 0.4830468	best: 0.4830468 (3)	total: 245ms	remaining: 5.88s
4:	learn: 0.4295737	test: 0.4784038	best: 0.4784038 (4)	total: 279ms	remaining: 5.31s
5:	learn: 0.4144896	test: 0.4745628	best: 0.4745628 (5)	total: 320ms	remaining: 5.02s
6:	learn: 0.3969836	test: 0.4713811	best: 0.4713811 (6)	total: 355ms	remaining: 4.72s
7:	learn: 0.3842524	test: 0.4740878	best: 0.4713811 (6)	total: 386ms	remaining: 4.44s
8:	learn: 0.3767217	test: 0.4691753	best: 0.4691753 (8)	total: 414ms	remaining: 4.18s
9:	learn: 0.3712488	test: 0.4523652	best: 0.4523652 (9)	total: 444ms	remaining: 3.99s
10:	learn: 0.3637330	test: 0.4543780	best: 0.4523652 (9)	total: 475ms	remaining: 3.84s
11:	learn: 0.3598068	test: 0.4573887	best: 0.4523652 

96:	learn: 0.1693287	test: 0.5904864	best: 0.4523652 (9)	total: 3.78s	remaining: 117ms
97:	learn: 0.1672291	test: 0.5951496	best: 0.4523652 (9)	total: 3.81s	remaining: 77.9ms
98:	learn: 0.1667795	test: 0.5952104	best: 0.4523652 (9)	total: 3.85s	remaining: 38.9ms
99:	learn: 0.1650422	test: 0.5964430	best: 0.4523652 (9)	total: 3.89s	remaining: 0us

bestTest = 0.4523651667
bestIteration = 9

Shrink model to first 10 iterations.


array([[0.47031199, 0.52968801],
       [0.60200615, 0.39799385],
       [0.67688946, 0.32311054],
       ...,
       [0.82959632, 0.17040368],
       [0.80118451, 0.19881549],
       [0.77210933, 0.22789067]])

In [36]:
resultado = auxiliarUnificarValor(res)
#resultado = res
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.52968801, 0.39799385, 0.32311054, ..., 0.17040368, 0.19881549,
       0.22789067])

In [37]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.53
10690,0.40
10691,0.32
10692,0.37
10693,0.83


In [38]:
importancia

,Feature Id,Importances
0,Product_Type,22.20
1,TRF,9.96
2,Planned_Delivery_Start_Date_dia,9.04
3,Region,8.51
4,Account_Created_Date_anio,5.50
5,Billing_Country,5.27
6,Account_Type,4.93
7,Last_Modified_Date_anio,3.91
8,Last_Modified_Date_dia,3.82
9,Total_Amount,3.77


## XGBoost

In [39]:
def xgboost(train_set, train_label,listaCategorica,test_set,masImportantes=0):
  xgb_reg = xgb.XGBClassifier(max_depth = 8, 
                           colsample_bytree = 0.5, 
                           learning_rate = 0.3, 
                           alpha = 8,
                           min_child_weight = 0.5,
                           n_estimators = 200,
                           objetive = 'reg:squarederror')
  xgb_reg.fit(train_set, train_label)
  predicts = xgb_reg.predict_proba(test_set)
  if (masImportantes > 0):
    xgb.plot_importance(xg_reg, max_num_features=masImportantes)
  return predicts




In [40]:
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B',
#                      'Account_Type','Delivery_Quarter','Billing_Country',
#                      'Delivery_Terms','Product_Family','ASP_(converted)_Currency']
#listaDeCategoricas = []
agrupacionEntrenamientoXGB = agrupacionEntrenamiento.copy(True)
agrupacionTesteoXGB = agrupacionTesteo.copy(True)

# convert categorical columns to integers 
#usando la funcion de catboost
for header in listaDeColumnasCategoricas:
  agrupacionEntrenamientoXGB[header] = agrupacionEntrenamientoXGB[header].astype('category').cat.codes
  agrupacionTesteoXGB[header] = agrupacionTesteoXGB[header].astype('category').cat.codes
#usando metodo personalizado
#for columna in listaDeCategoricas:
#  agrupacionEntrenamientoXGB,agrupacionTesteoXGB = convertirCategoryEnEnteros(agrupacionEntrenamientoXGB,agrupacionTesteoXGB,columna)


agrupacionEntrenamientoXGB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 27 columns):
modificaciones                      9791 non-null int64
Total_Amount                        9791 non-null float64
Region                              9791 non-null int8
Territory                           9791 non-null int8
ASP_(converted)                     9791 non-null float64
Account_Type                        9791 non-null int8
Delivery_Quarter                    9791 non-null int8
Product_Type                        9791 non-null int8
Billing_Country                     9791 non-null int8
Delivery_Terms                      9791 non-null int8
Product_Family                      9791 non-null int16
TRF                                 9791 non-null float64
Last_Modified_Date_anio             9791 non-null int64
Last_Modified_Date_mes              9791 non-null int64
Last_Modified_Date_dia              9791 non-null int64
Opportunity_Created_Date_anio       9791 non-null int

In [41]:
agrupacionTesteoXGB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 27 columns):
modificaciones                      1567 non-null int64
Total_Amount                        1567 non-null float64
Region                              1567 non-null int8
Territory                           1567 non-null int8
ASP_(converted)                     1567 non-null float64
Account_Type                        1567 non-null int8
Delivery_Quarter                    1567 non-null int8
Product_Type                        1567 non-null int8
Billing_Country                     1567 non-null int8
Delivery_Terms                      1567 non-null int8
Product_Family                      1567 non-null int8
TRF                                 1567 non-null float64
Last_Modified_Date_anio             1567 non-null int64
Last_Modified_Date_mes              1567 non-null int64
Last_Modified_Date_dia              1567 non-null int64
Opportunity_Created_Date_anio       1567 non-null 

In [42]:
res = xgboost(agrupacionEntrenamientoXGB,entrenamientoResultado,listaDeColumnasCategoricas,agrupacionTesteoXGB)
res

[21:15:15] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objetive } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




array([[0.28336102, 0.716639  ],
       [0.5818021 , 0.4181979 ],
       [0.71229357, 0.28770643],
       ...,
       [0.8123638 , 0.1876362 ],
       [0.38890254, 0.61109746],
       [0.76462305, 0.23537698]], dtype=float32)

In [43]:
resultado = auxiliarUnificarValor(res)
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.716639  , 0.41819787, 0.28770643, ..., 0.1876362 , 0.61109746,
       0.23537695], dtype=float32)

In [44]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.72
10690,0.42
10691,0.29
10692,0.23
10693,0.94


## LightGBM

In [45]:
def lightGBM(train_data,train_labels,cat_features,eval_data):    

      # Create training and validation sets

    x_train, x_test, y_train, y_test = train_test_split(train_data,train_labels, test_size=0.2, random_state=42, stratify=train_labels)

    categorical_features = cat_features
    train_data = lgb.Dataset(x_train, label=y_train, categorical_feature=categorical_features)
    test_data = lgb.Dataset(x_test, label=y_test)

     # Train the model
    parameters = {
          'application': 'binary',
         'objective': 'binary',
          'metric': 'auc',
          'is_unbalance': 'true',
          'boosting': 'gbdt',
          #'min_data_in_leaf': 200,
          'num_iteration': 122,
          'num_leaves': 31,
          'max_bin': 63,
          'feature_fraction': 0.05,
          'bagging_fraction': 0.05,
          'bagging_freq': 20,
          'learning_rate': 0.08,
          #'min_gain_to_split' : 5.0,
          'verbose': 0}

    model = lgb.train(parameters,
                        train_data,
                        valid_sets=test_data,
                        num_boost_round=5000,
                        early_stopping_rounds=100)

    submission = eval_data
    ids = submission.index.values

    x = submission
    y = model.predict(x)

    output = pd.DataFrame({'id': ids, 'target': y})
    output.to_csv("submission.csv", index=False)

    #lgb.plot_importance(model)

    return y

In [46]:
res = lightGBM(agrupacionEntrenamiento,entrenamientoResultado,listaDeColumnasCategoricas,agrupacionTesteo)
res

[1]	valid_0's auc: 0.630173
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.616402
[3]	valid_0's auc: 0.592844
[4]	valid_0's auc: 0.596706
[5]	valid_0's auc: 0.64264
[6]	valid_0's auc: 0.660348
[7]	valid_0's auc: 0.677374
[8]	valid_0's auc: 0.756495
[9]	valid_0's auc: 0.755831
[10]	valid_0's auc: 0.7734
[11]	valid_0's auc: 0.775419
[12]	valid_0's auc: 0.784384
[13]	valid_0's auc: 0.799289
[14]	valid_0's auc: 0.805283
[15]	valid_0's auc: 0.805586
[16]	valid_0's auc: 0.800138
[17]	valid_0's auc: 0.800591
[18]	valid_0's auc: 0.798589
[19]	valid_0's auc: 0.798748
[20]	valid_0's auc: 0.797448
[21]	valid_0's auc: 0.80369
[22]	valid_0's auc: 0.804962
[23]	valid_0's auc: 0.80657
[24]	valid_0's auc: 0.807589
[25]	valid_0's auc: 0.810687
[26]	valid_0's auc: 0.810219
[27]	valid_0's auc: 0.810215
[28]	valid_0's auc: 0.815152
[29]	valid_0's auc: 0.821339
[30]	valid_0's auc: 0.81947
[31]	valid_0's auc: 0.820387
[32]	valid_0's auc: 0.82134
[33]	valid_0's auc: 0.8234

array([0.58292946, 0.47724065, 0.50314643, ..., 0.22832029, 0.32203585,
       0.14111666])

In [49]:
resultado = pd.Series(res)

#ids = eval_data['Opportunity_ID'].values
def auxiliarUnificarValor1(arreglo):
    return arreglo[:]

resultado = auxiliarUnificarValor1(res)
resultado = (resultado - 1) * (-1)#En las ultimas pruebas es el opuesto el que genera mejor resultado
resultado

array([0.41707054, 0.52275935, 0.49685357, ..., 0.77167971, 0.67796415,
       0.85888334])

In [50]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.42
10690,0.52
10691,0.50
10692,0.60
10693,0.39


## Ramdom forest

### Bayesiano

In [ ]:
#Bayesian optimization
def bayesian_optimization(dataset, function, parameters):
   X_train, y_train, X_test, y_test = dataset
   n_iterations = 10
   gp_params = {"alpha": 1e-4}

   BO = BayesianOptimization(function, parameters)
   BO.maximize(n_iter=n_iterations, **gp_params)

   return BO.max

def rfc_optimization(cv_splits):
    def function(n_estimators, oob_score, min_samples_split):
        desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
        X_train = agrupacionEntrenamientoXGB.drop(desacerse,axis=1)
        y_train = entrenamientoResultado
        return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),
                   #max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)),
                   n_jobs=-1,
                   #random_state=42,
                   oob_score=(max(oob_score,1)),
                   bootstrap=True,
                   #class_weight="balanced"
                   max_features='sqrt'
                  ),
               X=X_train, 
               y=y_train, 
               cv=cv_splits,
               scoring="roc_auc",
               n_jobs=-1).mean()

    parameters = {"n_estimators": (200, 1000),
                  "oob_score": (5, 30),
                  "min_samples_split": (2, 10)}
    
    return function, parameters

#Train model
def ramdomForestBayesiano(X_train, y_train, X_test, y_test):
    dataset = (X_train, y_train, X_test, y_test)
    cv_splits = 4

    function, parameters = rfc_optimization(cv_splits)
    
    best_solution = bayesian_optimization(dataset, function, parameters)      
    params = best_solution["params"]

    model = RandomForestClassifier(
             n_estimators=int(max(params["n_estimators"], 0)),
             #max_depth=int(max(params["max_depth"], 1)),
             oob_score=(max('oob_score',1)),
             min_samples_split=int(max(params["min_samples_split"], 2)), 
             n_jobs=-1, 
             random_state=42,   
             class_weight="balanced")

    model.fit(X_train, y_train)
    predictions = model.predict_proba(X_test)
    return predictions

In [ ]:
#Uso los mismos valores de XGB para las variables categoricas ya que no las soporta
desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
#comentado para evitar ejecucion erronea
#res = ramdomForestBayesiano(agrupacionEntrenamientoXGB.drop(desacerse,axis=1),
#                            entrenamientoResultado,
#                            agrupacionTesteoXGB.drop(desacerse,axis=1),
#                            supuestos[['TargetBinario']])
#res

### Prediccion

In [ ]:
def ramdomForest(train_data,train_labels,eval_data):
  # Crear el modelo
  #Si no se le coloca un valor de max depth lo toma por min_samples_leaf a la creacion de hojas
  model = RandomForestClassifier(n_estimators=1000,
                                 #max_features=5,
                                 min_samples_leaf=9,
                                 oob_score=9,#7
                                 bootstrap=True,
                                 verbose=0,
                                 #max_leaf_nodes=20,
                                 #min_weight_fraction_leaf=0.01,
                                 max_features=6#7
                                 )
  #parametros importantes
  #  n_estimators: será la cantidad de árboles que generaremos.
  #  max_features: la manera de seleccionar la cantidad máxima de features para cada árbol.
  #  min_sample_leaf: número mínimo de elementos en las hojas para permitir un nuevo split (división) del nodo.
  #  oob_score: es un método que emula el cross-validation en árboles y permite mejorar la precisión y evitar overfitting.
  #  boostrap: para utilizar diversos tamaños de muestras para entrenar. Si se pone en falso, utilizará siempre el dataset completo.
  #  n_jobs: si tienes multiples cores en tu CPU, puedes indicar cuantos puede usar el modelo al entrenar para acelerar el entrenamiento.
  # a entrenar!
  model.fit(train_data, train_labels)
  predictions = model.predict_proba(eval_data)
  return predictions
  

In [ ]:
#Uso los mismos valores de XGB para las variables categoricas ya que no las soporta
desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes',
             'Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
agrupacionEntrenamientoRF = agrupacionEntrenamientoXGB.drop(desacerse,axis=1)
agrupacionTesteoRF = agrupacionTesteoXGB.drop(desacerse,axis=1)
#agrupacionTesteoRF = agrupacionTesteoXGB.drop(['Target'],axis=1)
print(len(agrupacionEntrenamientoRF.columns) - len(agrupacionTesteoRF.columns))

In [ ]:
res = ramdomForest(agrupacionEntrenamientoRF,
                   entrenamientoResultado,
                   agrupacionTesteoRF)
res

In [ ]:
resultado = auxiliarUnificarValor(res)
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

In [ ]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

## Perceptron

In [ ]:
for header in cat_features:
    agrupacionEntrenamiento[header] = agrupacionEntrenamiento[header].astype('category').cat.codes

In [ ]:
def perceptron(train_set, train_label, listaDeCategoricas, test_set):
  perceptron_mod = Perceptron(tol=1e-3, random_state=0)
  for header in listaDeCategoricas:
    train_set[header] = train_set[header].astype('category').cat.codes
  
  for header in listaDeCategoricas:
    test_set[header] = test_set[header].astype('category').cat.codes
  perceptron_mod.fit(train_set, train_label)
    
  predicts = perceptron_mod.predict(test_set)
  return predicts


In [ ]:
desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
agrupacionEntrenamientoPerceptron = agrupacionEntrenamiento.drop(desacerse,axis=1)
agrupacionTesteoPerceptron = agrupacionTesteo.drop(desacerse,axis=1)

In [ ]:
listaDeCategoricas = ['Region',
                      'Territory', 
                      'Product_Family', 
                      'Delivery_Terms', 
                      'Product_Type',
                      'Product_Category_B',
                      'Account_Type',
                      'Delivery_Quarter',
                      'Billing_Country',
                      'ASP_(converted)_Currency']
resPerceptron = perceptron(agrupacionEntrenamientoPerceptron, entrenamientoResultado, listaDeCategoricas, agrupacionTesteoPerceptron)
resPerceptron

### Predicción

In [ ]:
dataResultante = generarResultado(agrupacionTesteo,resPerceptron)
dataResultante.head()

## Multilayer Perceptron

In [54]:
def multiLayerPerceptron(train_set, train_label, listaDeCategoricas, test_set):
    n_neurons = [1028, 512, 256, 128, 64, 32, 16, 8, 4, 2, 1]
    multi_perceptron_mod = MLPRegressor(solver='lbfgs', 
                                hidden_layer_sizes=tuple(n_neurons[1:]),
                                max_iter=100000, 
                                shuffle=False,
                                random_state=9876,
                                activation='relu')
    
    for header in listaDeCategoricas:
        train_set[header] = train_set[header].astype('category').cat.codes
  
    for header in listaDeCategoricas:
        test_set[header] = test_set[header].astype('category').cat.codes
    
    multi_perceptron_mod.fit(train_set, train_label)
    multi_perceptron_mod.out_activation_ = 'logistic'
    predicts = multi_perceptron_mod.predict(test_set)
    
    return predicts

In [55]:
desacerse = ['Planned_Delivery_End_Date_anio','Planned_Delivery_End_Date_mes','Planned_Delivery_End_Date_dia','Last_Modified_Date_dia']
agrupacionEntrenamientoPerceptron = agrupacionEntrenamiento.drop(desacerse,axis=1)
agrupacionTesteoPerceptron = agrupacionTesteo.drop(desacerse,axis=1)

In [ ]:
agrupacionEntrenamientoPerceptron

In [56]:
one_hot_train_country = pd.get_dummies(agrupacionEntrenamientoPerceptron['Billing_Country'])
one_hot_test_country = pd.get_dummies(agrupacionTesteoPerceptron['Billing_Country'])


agrupacionEntrenamientoPerceptron = agrupacionEntrenamientoPerceptron.drop('Billing_Country',axis = 1)
agrupacionTesteoPerceptron = agrupacionTesteoPerceptron.drop('Billing_Country',axis = 1)

one_hot = pd.merge(one_hot_train_country, one_hot_test_country, on='Opportunity_ID',how='outer')


In [57]:
one_hot = one_hot.reset_index()

In [58]:
agrupacionEntrenamientoPerceptron = agrupacionEntrenamientoPerceptron.join(one_hot)
#agrupacionTesteoPerceptron = agrupacionTesteoPerceptron.reset_index()

In [60]:
#agrupacionEntrenamientoPerceptron = agrupacionEntrenamientoPerceptron.reset_index()
agrupacionTesteoPerceptron = agrupacionTesteoPerceptron.join(one_hot)

In [63]:
agrupacionEntrenamientoPerceptron.fillna(0, inplace=True)
agrupacionTesteoPerceptron.fillna(0, inplace=True)

In [ ]:
print(agrupacionEntrenamientoPerceptron.columns.tolist())

In [ ]:
print(agrupacionTesteoPerceptron.columns.tolist())

In [64]:
listaDeCategoricas = [
                      'Region',
                      'Territory', 
                      'Product_Family', 
                      'Delivery_Terms', 
                      'Product_Type',
                      'Account_Type',
                      'Delivery_Quarter',
                      #'Billing_Country'
                    ]
resMultiPerceptron = multiLayerPerceptron(agrupacionEntrenamientoPerceptron, entrenamientoResultado, listaDeCategoricas, agrupacionTesteoPerceptron)
#one_hot = pd.get_dummies(df['Territory'])
#agrupacionEntrenamientoPerceptron['Billing_Country'].unique()

In [ ]:
#Fixed result
resMultiPerceptron = (resMultiPerceptron - 1) * (-1)
resMultiPerceptron

### Predicción

In [65]:
dataResultante = generarResultado(agrupacionTesteo,resMultiPerceptron)